In [2]:
import sys
!{sys.executable} -m pip install pulp -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 17.7/17.7 MB 9.2 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pyb.exe -m pip install --upgrade pip


In [16]:
import pandas as pd
import pulp as pl



data_path = "Q2_未来30天各分拣中心每小时预测货量数据_方法2.xlsx"

data = pd.read_excel(data_path)




In [17]:



# 转换数据为更易处理的形式
data['班次'] = pd.cut(data['小时'], bins=[0, 5, 8, 12, 14, 16, 24], right=False, labels=[0, 1, 2, 3, 4, 5])
data_grouped = data.groupby(['分拣中心', '日期', '班次'])['预测每小时货量'].sum().reset_index()

# 分拣中心和班次的索引
centers = data_grouped['分拣中心'].unique()
days = data_grouped['日期'].unique()
shifts = data_grouped['班次'].unique()


In [19]:
import pulp as pl
# 定义问题
problem = pl.LpProblem("StaffingOptimization", pl.LpMinimize)

# 定义决策变量
x = pl.LpVariable.dicts("RegularStaff", (centers, days, shifts), lowBound=0, cat='Integer')
y = pl.LpVariable.dicts("TempStaff", (centers, days, shifts), lowBound=0, cat='Integer')

# 目标函数：最小化总人数
problem += pl.lpSum(x[c][d][s] + y[c][d][s] for c in centers for d in days for s in shifts)


In [20]:
# 货量处理约束
for c in centers:
    for d in days:
        for s in shifts:
            problem += 25 * x[c][d][s] + 20 * y[c][d][s] >= data_grouped[(data_grouped['分拣中心']==c) & (data_grouped['日期']==d) & (data_grouped['班次']==s)]['预测每小时货量'].sum()

# 正式工人数约束
for c in centers:
    for d in days:
        for s in shifts:
            problem += x[c][d][s] <= 60


In [ ]:
# 求解问题
problem.solve()

# 输出结果
results = []
for c in centers:
    for d in days:
        for s in shifts:
            results.append((c, d, s, pl.value(x[c][d][s]), pl.value(y[c][d][s])))

results_df = pd.DataFrame(results, columns=['分拣中心', '日期', '班次', '正式工人数', '临时工人数'])



In [22]:
results_df

,分拣中心,日期,班次,正式工人数,临时工人数
0,SC1,1,0,60.0,463.0
1,SC1,1,1,60.0,104.0
2,SC1,1,2,60.0,293.0
3,SC1,1,3,60.0,109.0
4,SC1,1,4,60.0,76.0
...,...,...,...,...,...
10255,SC9,30,1,60.0,50.0
10256,SC9,30,2,60.0,246.0
10257,SC9,30,3,9.0,0.0
10258,SC9,30,4,27.0,0.0


In [27]:
results_df[results_df['分拣中心']=='SC9'][results_df['日期']==30]

C:\Users\Stream\AppData\Local\Temp\ipykernel_17804\4230828664.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  results_df[results_df['分拣中心']=='SC9'][results_df['日期']==30]


,分拣中心,日期,班次,正式工人数,临时工人数
10254,SC9,30,0,60.0,386.0
10255,SC9,30,1,60.0,50.0
10256,SC9,30,2,60.0,246.0
10257,SC9,30,3,9.0,0.0
10258,SC9,30,4,27.0,0.0
10259,SC9,30,5,60.0,194.0


In [28]:
results_df.to_excel('Q3_表5输出.xlsx',index=None)

In [ ]:
import pandas as pd
import numpy as np

## 遗传算法求解
try:
    # 加载数据
    data = pd.read_excel("Q2_未来30天各分拣中心每小时预测货量数据_方法2.xlsx")

    # 将数据转换为按班次聚合的形式
    data['班次'] = pd.cut(data['小时'], bins=[0, 5, 8, 12, 14, 16, 24], right=False, labels=[0, 1, 2, 3, 4, 5])
    data_grouped = data.groupby(['分拣中心', '日期', '班次'])['预测每小时货量'].sum().reset_index()

    # 获取唯一的中心、日期和班次
    centers = data_grouped['分拣中心'].unique()
    days = data_grouped['日期'].unique()
    shifts = data_grouped['班次'].unique()

    # 定义适应度函数
    def fitness_function(solution):
        # 将解码的基因型映射到工作人员数量
        x = {}
        y = {}
        idx = 0
        for c in centers:
            for d in days:
                for s in shifts:
                    x[(c, d, s)] = solution[idx]
                    idx += 1
        for c in centers:
            for d in days:
                for s in shifts:
                    y[(c, d, s)] = solution[idx]
                    idx += 1

        # 计算总工人数
        total_workers = sum(solution)

        # 计算约束条件是否满足
        constraints_satisfied = True
        for c in centers:
            for d in days:
                for s in shifts:
                    required_quantity = data_grouped[(data_grouped['分拣中心'] == c) & (data_grouped['日期'] == d) & (data_grouped['班次'] == s)]['预测每小时货量'].sum()
                    if 25 * x[(c, d, s)] + 20 * y[(c, d, s)] < required_quantity:
                        constraints_satisfied = False
                        break

        # 如果约束条件不满足，适应度为一个极大值
        if not constraints_satisfied:
            return 1e6

        # 否则，适应度为总工人数
        return total_workers

    # 定义遗传算法参数
    population_size = 100
    num_generations = 100
    num_genes = len(centers) * len(days) * len(shifts) * 2

    # 初始化种群
    population = np.random.randint(0, 61, size=(population_size, num_genes))  # 每个变量的范围为0到60

    # 运行遗传算法
    for generation in range(num_generations):
        # 计算适应度
        fitness_values = [fitness_function(individual) for individual in population]

        # 选择父代
        parents = population[np.argsort(fitness_values)[:population_size // 2]]

        # 生成子代
        offspring = []
        while len(offspring) < population_size:
            parent1, parent2 = np.random.choice(parents, size=2, replace=False)
            crossover_point = np.random.randint(1, num_genes)
            child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
            child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
            offspring.append(child1)
            offspring.append(child2)

        # 变异
        mutation_rate = 0.1
        for individual in offspring:
            if np.random.rand() < mutation_rate:
                gene_to_mutate = np.random.randint(num_genes)
                individual[gene_to_mutate] = np.random.randint(0, 61)

        # 更新种群
        population = np.array(offspring)

    # 获取最优解
    best_solution_idx = np.argmin([fitness_function(individual) for individual in population])
    best_solution = population[best_solution_idx]

    # 将最优解映射回人员数量
    x = {}
    y = {}
    idx = 0
    for c in centers:
        for d in days:
            for s in shifts:
                x[(c, d, s)] = best_solution[idx]
                idx += 1
    for c in centers:
        for d in days:
            for s in shifts:
                y[(c, d, s)] = best_solution[idx]
                idx += 1

    # 构建结果 DataFrame
    results = []
    for c in centers:
        for d in days:
            for s in shifts:
                results.append((c, d, s, x[(c, d, s)], y[(c, d, s)]))

    results_df = pd.DataFrame(results, columns=['分拣中心', '日期', '班次', '正式工人数', '临时工人数'])
    results_df
except:
    results_df


In [ ]:
results_df.to_excel('Q3遗传算法_表5输出.xlsx',index=None)